In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input, Lambda

def generate_dataset(num_examples, model):
  """Generate a dataset of Tic-Tac-Toe boards and labels"""
  X = np.zeros((num_examples, 9))
  y = np.zeros((num_examples, 9))
  for i in range(num_examples):
    # Generate a random Tic-Tac-Toe board
    board = np.random.randint(0, 3, size=(3, 3))
    X[i] = board.flatten()
    # Calculate the label for the board
    if np.any(np.sum(board, axis=0) == 3) or np.any(np.sum(board, axis=1) == 3) or np.sum(np.diag(board)) == 3 or np.sum(np.diag(np.fliplr(board))) == 3:
      # player 1 wins
      y[i] = np.zeros(9)
    elif np.any(np.sum(board, axis=0) == -3) or np.any(np.sum(board, axis=1) == -3) or np.sum(np.diag(board)) == -3 or np.sum(np.diag(np.fliplr(board))) == -3:
      # player 2 wins
      y[i] = np.zeros(9)
    elif np.any(board == 0):
      # find the optimal move for the current player
      y[i] = model.predict(np.expand_dims(board.flatten(), axis=0), current_player=1 if np.sum(board) == 1 else 2)[0]
      y[i] = np.eye(9)[np.argmax(y[i])]
    else:
      # draw
      y[i] = np.zeros(9)
  return X, y

# Define a neural network model
model = Sequential()
model.add(Dense(32, input_dim=9, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(9, activation='softmax'))

# Define the input layer
inputs = Input(shape=(9,))

# Define the lambda layer
outputs = Lambda(lambda x, current_player=1: model(x, training=False, current_player=current_player))(inputs)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Generate a dataset of 1000 Tic-Tac-Toe boards
num_examples = 1000
X, y = generate_dataset(num_examples, model)

# Shuffle the dataset
indices = np.arange(num_examples)
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

# Reshape the input data to have shape (num_examples, 9)
X = X.reshape((num_examples, 9))

# Train the model
model.fit(X, y, epochs=10)

def print_board(board):
  """Print the Tic-Tac-Toe board"""
  print("  0 1 2")
  for i in range(3):
    print(i, end=" ")
    row = " ".join(["X" if cell == 1 else "O" if cell == 2 else " " for cell in board[i]])
    print(row)

def get_move(board, player):
  """Prompt the user for a row and column and return the move as a tuple (row, col)"""
  if player == 1:
    player_marker = "X"
  else:
    player_marker = "O"
  while True:
    try:
      row = int(input(f"Player {player_marker}, enter a row (0, 1, or 2): "))
      col = int(input(f"Player {player_marker}, enter a column (0, 1, or 2): "))
      if row < 0 or row > 2 or col < 0 or col > 2:
        print("Invalid move. Please try again.")
      elif board[row, col] != 0:
        print("That cell is already occupied. Please try again.")
      else:
        return (row, col)
    except ValueError:
      print("Invalid input. Please try again.")

def check_win(board, player):
  """Check if the given player has won the game"""
  # Check rows
  for i in range(3):
    if np.all(board[i] == player):
      return True
  # Check columns
  for i in range(3):
    if np.all(board[:, i] == player):
      return True
  # Check diagonals
  if np.all(np.diag(board) == player) or np.all(np.diag(np.fliplr(board)) == player):
    return True
  return False

def check_draw(board):
  """Check if the game is a draw (i.e., the board is full and no one has won)"""
  return np.all(board != 0) and not check_win(board, 1) and not check_win(board, 2)

def play_game(model):
  """Play a game of Tic-Tac-Toe against the AI"""
  board = np.zeros((3, 3))
  player = 1
  while True:
    # Print the current board
    print_board(board)
    if player == 1:
      # Human player's turn
      move = get_move(board)
      board[move] = 1
      if check_win(board, player):
        print("You won!")
        break
      elif check_draw(board):
        print("It's a draw!")
        break
      player = 2
    else:
      # AI player's turn
      move = np.argmax(model.predict(np.expand_dims(np.hstack((board.flatten(), player)), axis=0))[0])
      row = move // 3
      col = move % 3
      board[row, col] = 2
      if check_win(board, player):
        print_board(board)
        print("AI won!")
        break
      elif check_draw(board):
        print("It's a draw!")
        break
      player = 1

play_game(model)

TypeError: ignored